In [2]:
import numpy as np
import cv2
from scipy.spatial.transform import Rotation as R
%config Completer.use_jedi = False

In [12]:
# Create ground truth tag locations, general camera locations, and camera 0-3 locations

# create the ground truth tag locations in the world coordinate system
# The april tags are at the 20 locations given in the 4x20 array A (in homogeneous coordinates)
# The tags are placed on both sides of a 6m wide corridor along the world x axis, half at elevation 0m
# the other half at elevation 1m
A = np.array([4*list(range(0,5)),
              10*[3] + 10*[-3],
              2*(5*[0] + 5*[1]),
              20*[1]])


# Create the generalized camera locations for the simulated data
# The generalized camera starts out at the world cooridnate frame and moves along the world y axis
# and moves sinusoidally along x
numframes = 15
rotperframe = np.pi/12.0
xtransperframe = 4.0/numframes
ytransamp = 1
ytransfreq = 2*np.pi/numframes     # one cycle over full path
T = np.zeros((numframes, 4,4))
for f in range(numframes):
    T[f, 0:3,0:3] = R.from_rotvec([0,0,f*rotperframe]).as_matrix()
    T[f, :,3] = [f*xtransperframe, ytransamp*np.cos(ytransfreq*f), 0, 1]
# print(np.round(T, decimals=4))


# Create camera 0-3 locations in the general camera frame as an array CT. CT[i,:,:] is the transform 
# from camera i to the generalized camera frame. The generalized camera origin is at the center of the
# four cameras with x forward, y to the left, and z up (relative to the robot). The real camera 
# origins are on the corners of a 60cm x 10cm (in y and x of the general camera) rectangle 
# centered on the general camera origin and in the general camera z=0 plane. The real cameras are
# numbered as follows:
#
#                                           front of robot
#          2 ----------------------------------------------------------------------------- 0
#           |                                    x ^                                      |
#           |                                      |                                      |
#           |                               y      |                                      |
#   10 cm   |                              <-------o                                      |
#           |                                                                             |
#           |                                                                             |
#           |                                                                             |
#          3 ----------------------------------------------------------------------------- 1
#                                                60 cm
#
# The angles of the camera viewpoints in the horizontal plane measured CCW from robot forward 
# direction
camangle0 = -  np.pi/4  # - 37.5 * np.pi/180.0
camangle1 = -3*np.pi/4  # -157.5 * np.pi/180.0
camangle2 =    np.pi/4  #   37.5 * np.pi/180.0
camangle3 =  3*np.pi/4  #  157.5 * np.pi/180.0
# The real camera x-z planes are coincident with the general camera x-y plane. 
CT = np.zeros((4, 4, 4))
CT[0, 0:3,0:3] = (R.from_rotvec([-np.pi/2,0,0]) * R.from_rotvec([0, np.pi/2 - camangle0, 0])).as_matrix()
CT[0, :,3] = [ 0.05, -0.3, 0, 1]
CT[1,0:3,0:3] = (R.from_rotvec([-np.pi/2,0,0]) * R.from_rotvec([0, np.pi/2 - camangle1, 0])).as_matrix()
CT[1, :,3] = [-0.05, -0.3, 0, 1]
CT[2,0:3,0:3] = (R.from_rotvec([-np.pi/2,0,0]) * R.from_rotvec([0, np.pi/2 - camangle2, 0])).as_matrix()
CT[2, :,3] = [ 0.05,  0.3, 0, 1]
CT[3,0:3,0:3] = (R.from_rotvec([-np.pi/2,0,0]) * R.from_rotvec([0, np.pi/2 - camangle3, 0])).as_matrix()
CT[3, :,3] = [-0.05,  0.3, 0, 1]

# print the computed transforms for cameras 0-3
# for ct in CT:
#     print(np.round(ct, decimals=4))

# print the camera 
# for a in range(15, 360, 30):
#     Gs = 3*np.array([np.cos(a*np.pi/180.0), np.sin(a*np.pi/180.0), 0])
#     print(Gs[0:3].shape, CT[0, 0:3, 2].shape)
#     unit = lambda v: v / np.linalg.norm(v)    # normalize vector to unit length
#     camera = np.argmax([np.dot(unit(Gs[0:3] - ct[0:3,3]), ct[0:3, 2]) for ct in CT])
#     print('for angle ', a, ' select camera ', camera)

# Project features to produce generalized camera ray directions P and ray origins O. 
# Assume tags project into the real cameras 0 to (k-1) whose z axis is closest to the 
# direction to tag.
def project(T, A, CT, camera_params=None):
    """
       T is the transform from the generalized camera to the world
       A is a 4 by n array with the homogeneous coordinates of the tags
       CT is a k by 4 by 4 array of k Euclidean transforms into the k real cameras
       camera_params is a length k array of tuples (camera_matrix, distortion_coeffs),
                    one for each real camera
       
       Returns:
       O: n by 3 array of the projection ray origins in the generalized camera frame
       P: n by 3 array of the projection ray directions (to the tags) in the generalized camera frame
       UV: length n list of (camera number, projected image points) tuples
    """
    n = A.shape[1]
    k = CT.shape[0]
    G = np.linalg.inv(T) @ A   # tag locations in generalized camera frame
    O = np.zeros((n,3))
    P = np.zeros((n,3))
    UV = n * [None]
    unit = lambda v: v / np.linalg.norm(v)    # normalize vector to unit length
    for p in range(n):
        # select the camera whose z-axis direction is closest to the direction to the tag 
        camera = np.argmax([np.dot(unit(G[0:3,p] - ct[0:3, 3]), ct[0:3, 2]) for ct in CT])
        O[p,:] = CT[camera, 0:3, 3]
        # transform the tag in generalized camera coords to the real camera coords, then
        # rotate this direction back to generalized camera coords again and normalize to unit length
        P[p,:] = G[0:3, p] - CT[camera, 0:3, 3]
        P[p,:] /= np.linalg.norm(P[p,:])
        
        if camera_params is not None:
            uv, _ = cv2.projectPoints((np.linalg.inv(CT[camera,:,:]) @ G[:,p])[0:3], 
                                       np.zeros((1,3)), np.zeros((1,3)),
                                      *camera_params[camera])
            UV[p] = (camera, uv[0].T)
        
    return P, O, UV



In [13]:
cammat = lambda f, w, h: np.array([[f, 0, w/2],[0, f, h/2],[0,0,1]])
camera_params = ((cammat(500, 1024, 1024), np.array([])),
                 (cammat(400, 800, 600), np.array([])),
                 (cammat(350, 1024, 1024), np.array([])),
                 (cammat(500, 1280, 1600), np.array([])))
P, O, UV = project(T[0], A, CT, camera_params)

In [14]:
UV

[(2,
  array([[140.78787879],
         [512.        ]])),
 (2,
  array([[412.94339623],
         [512.        ]])),
 (2,
  array([[535.97260274],
         [512.        ]])),
 (2,
  array([[606.08602151],
         [512.        ]])),
 (2,
  array([[651.38053097],
         [512.        ]])),
 (2,
  array([[140.78787879],
         [212.01530495]])),
 (2,
  array([[412.94339623],
         [325.21707667]])),
 (2,
  array([[535.97260274],
         [376.39048032]])),
 (2,
  array([[606.08602151],
         [405.55381789]])),
 (2,
  array([[651.38053097],
         [424.39385012]])),
 (1,
  array([[-10.95890411],
         [300.        ]])),
 (0,
  array([[807.69892473],
         [512.        ]])),
 (0,
  array([[666.86725664],
         [512.        ]])),
 (0,
  array([[568.39097744],
         [512.        ]])),
 (0,
  array([[495.66013072],
         [512.        ]])),
 (1,
  array([[-10.95890411],
         [145.01769179]])),
 (0,
  array([[807.69892473],
         [359.93402555]])),
 (0,
  array([

In [16]:
for i, ppt in enumerate(UV):
    camera = ppt[0]
    pt = ppt[1]
    dir = cv2.undistortPoints(pt, *camera_params[camera])
    dir.shape
    dir = CT[camera,0:3,0:3] @ np.array([[dir[0,0,0]],[dir[0,0,1]],[1.0]])
    print(np.round(dir.T/np.linalg.norm(dir), decimals=4),
          np.round(P[i,:], decimals=4))

[[-0.0294  0.9996 -0.    ]] [-0.0294  0.9996  0.    ]
[[0.4878 0.8729 0.    ]] [0.4878 0.8729 0.    ]
[[0.7538 0.6571 0.    ]] [0.7538 0.6571 0.    ]
[[0.8664 0.4993 0.    ]] [0.8664 0.4993 0.    ]
[[0.9185 0.3953 0.    ]] [0.9185 0.3953 0.    ]
[[-0.0253  0.8617  0.5069]] [-0.0253  0.8617  0.5069]
[[0.434  0.7765 0.4568]] [0.434  0.7765 0.4568]
[[0.7031 0.6129 0.3606]] [0.7031 0.6129 0.3606]
[[0.8313 0.4791 0.2818]] [0.8313 0.4791 0.2818]
[[0.8947 0.385  0.2265]] [0.8947 0.385  0.2265]
[[ 0.0135 -0.9999 -0.    ]] [ 0.0135 -0.9999  0.    ]
[[ 0.2487 -0.9686  0.    ]] [ 0.2487 -0.9686  0.    ]
[[ 0.4662 -0.8847 -0.    ]] [ 0.4662 -0.8847  0.    ]
[[ 0.6234 -0.7819 -0.    ]] [ 0.6234 -0.7819  0.    ]
[[ 0.7298 -0.6836  0.    ]] [ 0.7298 -0.6836  0.    ]
[[ 0.013  -0.9653  0.2609]] [ 0.013  -0.9653  0.2609]
[[ 0.2406 -0.937   0.2532]] [ 0.2406 -0.937   0.2532]
[[ 0.4535 -0.8604  0.2325]] [ 0.4535 -0.8604  0.2325]
[[ 0.6099 -0.765   0.2068]] [ 0.6099 -0.765   0.2068]
[[ 0.7177 -0.6723  0.1

In [31]:
P

array([[-2.93990516e-02,  9.99567754e-01,  1.44948515e-32],
       [ 4.87821358e-01,  8.72943482e-01,  2.53173156e-32],
       [ 7.53772730e-01,  6.57135200e-01,  0.00000000e+00],
       [ 8.66429965e-01,  4.99298624e-01,  1.44807781e-32],
       [ 9.18542556e-01,  3.95322113e-01,  0.00000000e+00],
       [-2.53428640e-02,  8.61657377e-01,  5.06857281e-01],
       [ 4.33952850e-01,  7.76547204e-01,  4.56792473e-01],
       [ 7.03073710e-01,  6.12936055e-01,  3.60550621e-01],
       [ 8.31315803e-01,  4.79063344e-01,  2.81801967e-01],
       [ 8.94670926e-01,  3.85048246e-01,  2.26498969e-01],
       [ 1.35122798e-02, -9.99908705e-01,  1.99862049e-32],
       [ 2.48690247e-01, -9.68583069e-01,  2.58134228e-32],
       [ 4.66239077e-01, -8.84658761e-01,  1.17883904e-32],
       [ 6.23405636e-01, -7.81898595e-01,  0.00000000e+00],
       [ 7.29825236e-01, -6.83633765e-01,  0.00000000e+00],
       [ 1.30443413e-02, -9.65281259e-01,  2.60886827e-01],
       [ 2.40583439e-01, -9.37009184e-01